#Final Ensemble Model

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/Challenge1


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Challenge1


In [ ]:
dataset_dir = 'training_data_final'

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Random seed for reproducibility
seed = 1270
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

load the models

In [ ]:
vgg16 = tfk.models.load_model("model_vgg16/CNN_vgg16Best")
efficientnetb6 = tfk.models.load_model("effnet6/effnet6_ftl")
#xception = tfk.models.load_model("xception/xception89_colab")

In [ ]:
models = []

models.append(vgg16)
models.append(efficientnetb6)
#models.append(mymodel)
# models.append(xception)

In [ ]:
vgg16._name = "vgg16"
efficientnetb6._name = "eff6"
# mymodel._name = "mine"
# xception._name ='xception'

In [ ]:
#function to collect models outputs and to average them
def ensembleModels(models, model_input):
  
    Models=[model(model_input) for model in models] 

    Avg=tfk.layers.average(Models) 
    
    modelEns = tfk.Model(inputs=model_input, outputs=Avg, name='ensemble')  
   
    return modelEns

In [ ]:
model_input = tfk.Input(shape=models[0].input_shape[1:]) # c*h*w
ensemble_model = ensembleModels(models, model_input)
ensemble_model.compile(optimizer=tfk.optimizers.Adam(),
              loss=tfk.losses.CategoricalCrossentropy(),
              metrics='accuracy')
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 vgg16 (Functional)             (None, 8)            14913864    ['input_14[0][0]']               
                                                                                                  
 eff6 (Functional)              (None, 8)            42407063    ['input_14[0][0]']               
                                                                                                  
 average_9 (Average)            (None, 8)            0           ['vgg16[9][0]',                  
                                                                  'eff6[4][0]']            

In [ ]:
ensemble_model = tfk.models.load_model("EnsembleModel1")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch = 32

data_gen = ImageDataGenerator(validation_split=0.15,rotation_range=10, height_shift_range=10, width_shift_range=10,zoom_range=0.15,horizontal_flip=True,  vertical_flip=True,  fill_mode='reflect') 

train_gen = data_gen.flow_from_directory(directory=dataset_dir,target_size=(96,96),color_mode='rgb', classes=None, class_mode='categorical',batch_size=batch,shuffle=True,seed=seed,subset='training') # set as training data

val_gen = data_gen.flow_from_directory(directory=dataset_dir,target_size=(96,96),color_mode='rgb',classes=None,class_mode='categorical',batch_size=batch, shuffle=True, seed=seed, subset='validation') # set as validation data

This evaulation is distorted since we have trained the single models on different subsets of the training set

In [ ]:
# Evaluate on validation set

model_test_metrics = modelEns.evaluate(val_gen, return_dict=True)

print()
print(model_test_metrics)

17/17 [==============================] - 11s 169ms/step - loss: 0.1766 - accuracy: 0.9620

Test metrics without data augmentation
{'loss': 0.17658136785030365, 'accuracy': 0.9620493650436401}


In [ ]:
# ensemble_model.save("EnsembleModel1")